# Convert raw EEG data to BIDS

This script will take raw EEG data files stored in the `sourcedata` folder (as saved by OpenVibe, PyCorder, NeStation, or ASAlab (after conversion to EEGLAB format)) and convert them to BIDS-compatible files and folder structure (saved to `rawdata`), with all metadata.

It should require little to no modification for individual studies.

You must save the raw EEG files in the `sourcedata/sub-00x/eeg` subfolder of your study's BIDS root folder.

All study-specific configuration details should be defined in the `config.yml` file and not in this script. 

The only thing to change in this script is to list the subject IDs of the subjects whose data you wish to convert, in the cell below. Then just run all additional cells.


---
(c) 2023 Aaron J. Newman, NeuroCogntive Imaging Lab, Dalhousie University

Released under a [CC BY 4.0](https://creativecommons.org/licenses/by/4.0/) license.

---

In [ ]:
from os import path as op
import os
import json
# import configparser
import yaml
try:
    from yaml import CLoader as Loader, CDumper as Dumper
except ImportError:
    from yaml import Loader, Dumper

from pyxdf import resolve_streams
from mnelab.io.xdf import read_raw_xdf

import shutil
from glob import glob
from pathlib import Path
import random

import numpy as np 
# from matplotlib import pyplot as plt
import mne
# from mne.datasets import eegbci
mne.set_log_level('error')

from mne_bids import write_raw_bids, BIDSPath, update_sidecar_json
from mne_bids.stats import count_events

## Study Parameters

Will import study-level parameters from `config.yml` in `bids_root`

In [ ]:
# this shouldn't change if you run this script from its default location in code/import
bids_root = '../..'

cfg_file = op.join(bids_root, 'config.yml')
with open(cfg_file, 'r') as f:
    config = yaml.load(f, Loader=Loader)

eog = {k: v for d in eog for k, v in d.items()}
montage_fname = config['montage_fname']
raw_extn = config['raw_extn']

## Paths

In [ ]:
# source_path is where the input source (raw) files live
source_path = op.join(bids_root, 'sourcedata')

# raw_path is where the results of running this script will be saved
raw_path = op.join(bids_root)                  

In [ ]:
# convert all participants in sourcedata
prefix = 'sub-'
in_subjs = sorted([s[-7:] for s in glob(source_path + '/' + prefix + '*')])

in_subjs

## Import data, and covert to BIDS

In [ ]:
for subject in in_subjs:
    print(subject)
    sessions = [f.split('/')[-1][-3:] for f in glob(op.join(source_path, subject, 'ses-*'))]

    for session in sessions:
        print('Session:', session)
        in_files = glob(op.join(source_path, subject, 'ses-' + session, data_type, subject + '*' + raw_extn))
        
        for idx, f in enumerate(in_files):
            print(f)
            bids_parts = f.split('/')[-1].split('_')
            task = [t for t in bids_parts if 'task' in t][0].split('-')[-1]
            run = [r for r in bids_parts if 'run' in r][0][-2:]

            # EEGLAB format
            if eeg_extn == 'set':
                    raw = mne.io.read_raw_eeglab(f,  
                                                preload=False).set_montage(montage_fname)

            # EGI format
            elif eeg_extn == 'raw':    
                raw = mne.io.read_raw_egi(f, 
                                          preload=False, 
                                          misc=['E129'],  
                                          exclude=[], 
                                      ).set_montage(montage_fname)

            # BrainVision format    
             elif eeg_extn == 'vhdr':
                raw = mne.io.read_raw_brainvision(f, 
                                                  preload=False, 
                                                  misc=['Digi']).set_montage(montage_fname)

            # XDF format
            elif eeg_extn == 'xdf':
                streams = resolve_streams(f)
                eeg_stream_idx = [idx for idx, stream in enumerate(streams) 
                                if stream['name'] == amp_name][0]
                eeg_stream_id = [streams[eeg_stream_idx]['stream_id']]
                srate = streams[eeg_stream_idx]['nominal_srate']
                raw = read_raw_xdf(f, 
                                eeg_stream_id, 
                                fs_new=srate).set_montage('easycap-M1')
                
            else:
                print('Unknown file format:', raw_extn)
                
            # rename and label EOG channels
            raw.rename_channels(eog)
            for ch in eog.values():
                raw.set_channel_types({ch:'eog'})

            # --- Make BIDS ---
            bids_path = BIDSPath(subject=subject[-3:], 
                                task=task, 
                                session=session,
                                run=run,
                                datatype=data_type,
                                suffix="eeg",
                                root=op.join(raw_path)
                                )
                                
            bids_path.rm(safe_remove=False)

            write_raw_bids(raw, bids_path, 
                        anonymize={'daysback':random.randint(35452, 40000)},
                        format=out_format,
                        overwrite=True,
                        allow_preload=True
                        )

            # Update metadata with additional info
            entries = { 'Manufacturer':config['amp_mfr'],
                        'ManufacturersModelName':config['amp_model'],
                        'PowerLineFrequency':config['line_freq'],
                        'SoftwareFilters':config['sw_filt'],
                        'HardwareFilters':config['hw_filt'],
                        'EEGReference':config['eeg_ref'],
                        'EEGGround':config['eeg_ground'],
                        'CapManufacturer':config['cap_mfr'],
                        'CapManufacturersModelName':config['cap_model'],
                        'EthicsApprovals':config['reb_info'],
                        'InstitutionName':config['instn'],
                        'InstitutionAddress':config['instn_addr'],
                        'InstitutionalDepartmentName':config['instn_dept'],
                        }

            sidecar_path = bids_path.copy().update(extension='.json')
            update_sidecar_json(bids_path=sidecar_path, entries=entries)

# Merge README file generated by write_raw_bids with the eixting README.md        
with open(raw_path + '/README') as fi:
    lines = fi.read().splitlines()    

with open(raw_path + '/README.md', 'a') as fo:
    for li in lines:
        fo.write('\n')
        fo.write(li)        

os.remove(raw_path + '/README')        